# Gépi Tanulás Klasszifikációs Feladatok Összefoglalója

Ez a notebook összefoglalja a két klasszifikációs feladatot:
1. Osztályozás szintetikus adatokon.
2. Osztályozás a Wine adathalmazon.

## Szükséges Könyvtárak Importálása

Először importáljuk az összes szükséges könyvtárat mindkét feladathoz.

In [ ]:
import numpy as np
import pandas as pd
import time
import os
import warnings

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import ConvergenceWarning

import matplotlib.pyplot as plt
import seaborn as sns

# Mappák létrehozása az ábráknak (ha még nem léteznek)
output_dir_part1 = "part1_plots"
output_dir_part2 = "part2_plots"
if not os.path.exists(output_dir_part1):
    os.makedirs(output_dir_part1)
if not os.path.exists(output_dir_part2):
    os.makedirs(output_dir_part2)

# Figyelmeztetések alapértelmezett kezelése
warnings.filterwarnings("default", category=ConvergenceWarning)

# 1. Feladat: Osztályozás Szintetikus Adatokon

## 1.1 Adatgenerálás

Létrehozunk egy mesterséges adathalmazt `make_blobs` segítségével. Az adathalmaz több dimenziós, de jól elkülönülő csoportokat (blobokat) tartalmaz.

In [ ]:
n_samples = 241
n_features = 11
centers = 4
cluster_std = 0.6
random_state_part1 = 3650653441

X_part1, y_part1 = make_blobs(n_samples=n_samples,
                             n_features=n_features,
                             centers=centers,
                             cluster_std=cluster_std,
                             random_state=random_state_part1)

print(f"1. Feladat - Generált adathalmaz alakja: X={X_part1.shape}, y={y_part1.shape}")
print(f"1. Feladat - Osztályok eloszlása: {np.bincount(y_part1)}")

## 1.2 Adat Vizualizációja (PCA)

Mivel az adathalmaz több mint 2 dimenziós, Főkomponens Analízist (PCA) használunk a dimenziócsökkentésre, hogy 2D-ben ábrázolhassuk az osztályokat. PCA előtt standardizáljuk az adatokat.

In [ ]:
scaler_pca_part1 = StandardScaler()
X_scaled_part1 = scaler_pca_part1.fit_transform(X_part1)

pca_part1 = PCA(n_components=2, random_state=random_state_part1)
X_pca_part1 = pca_part1.fit_transform(X_scaled_part1)

plt.figure(figsize=(8, 6))
scatter_part1 = plt.scatter(X_pca_part1[:, 0], X_pca_part1[:, 1], c=y_part1, cmap='viridis', edgecolor='k', s=50)
plt.title('1. Feladat - Generált adathalmaz (PCA után)')
plt.xlabel('Első főkomponens')
plt.ylabel('Második főkomponens')
plt.legend(handles=scatter_part1.legend_elements()[0], labels=[f'Osztály {i}' for i in range(centers)])
plt.grid(True)
plt.savefig(os.path.join(output_dir_part1, 'generated_data_pca.png'))
plt.show()

## 1.3 Adat Felosztása és Skálázása

Az eredeti (nem PCA-zott) adatokat tanító és teszt halmazra osztjuk. A modellek (különösen az SVM és LogReg) tanítása előtt a tanító adatokon illesztett `StandardScaler`-rel skálázzuk a jellemzőket.

In [ ]:
X_train_part1, X_test_part1, y_train_part1, y_test_part1 = train_test_split(
    X_part1, y_part1, test_size=0.3, random_state=random_state_part1, stratify=y_part1
)

scaler_model_part1 = StandardScaler()
X_train_scaled_part1 = scaler_model_part1.fit_transform(X_train_part1)
X_test_scaled_part1 = scaler_model_part1.transform(X_test_part1)

print(f"1. Feladat - Tanító halmaz mérete: {X_train_scaled_part1.shape[0]}")
print(f"1. Feladat - Teszt halmaz mérete: {X_test_scaled_part1.shape[0]}")

## 1.4 Modellek Tanítása és Kiértékelése

Definiáljuk, tanítjuk és kiértékeljük a Naiv Bayes, SVM és Logisztikus Regresszió modelleket. Mérjük a tanítási időt, pontosságot, és elkészítjük a riportot, valamint a konfúziós mátrixot.

In [ ]:
models_part1 = {
    "Naive Bayes": GaussianNB(),
    "Support Vector Machine": SVC(random_state=random_state_part1, probability=True),
    "Logistic Regression": LogisticRegression(random_state=random_state_part1, max_iter=1000)
}

results_part1 = {}
training_times_part1 = {}

print("\n--- 1. Feladat: Modellek Kiértékelése ---")

for name, model in models_part1.items():
    print(f"\n--- {name} ---")
    start_time = time.time()
    model.fit(X_train_scaled_part1, y_train_part1)
    end_time = time.time()
    training_times_part1[name] = end_time - start_time

    y_pred_train_part1 = model.predict(X_train_scaled_part1)
    y_pred_test_part1 = model.predict(X_test_scaled_part1)

    accuracy_train_part1 = accuracy_score(y_train_part1, y_pred_train_part1)
    accuracy_test_part1 = accuracy_score(y_test_part1, y_pred_test_part1)
    report_test_part1 = classification_report(y_test_part1, y_pred_test_part1)
    cm_test_part1 = confusion_matrix(y_test_part1, y_pred_test_part1)

    results_part1[name] = {
        "model": model,
        "accuracy_train": accuracy_train_part1,
        "accuracy_test": accuracy_test_part1,
        "report_test": report_test_part1,
        "confusion_matrix_test": cm_test_part1
    }

    print(f"Tanítási idő: {training_times_part1[name]:.4f} másodperc")
    print(f"Pontosság (Tanító halmaz): {accuracy_train_part1:.4f}")
    print(f"Pontosság (Teszt halmaz): {accuracy_test_part1:.4f}")
    print("Osztályozási riport (Teszt halmaz):")
    print(report_test_part1)
    print("Konfúziós mátrix (Teszt halmaz):")

    # Konfúziós mátrix ábrázolása
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm_test_part1, annot=True, fmt='d', cmap='Blues', xticklabels=range(centers), yticklabels=range(centers))
    plt.xlabel('Jósolt osztály')
    plt.ylabel('Valós osztály')
    plt.title(f'{name} - Konfúziós Mátrix (Teszt)')
    safe_name = name.replace(" ", "_").lower()
    plt.savefig(os.path.join(output_dir_part1, f'confusion_matrix_{safe_name}.png'))
    plt.show()

## 1.5 Eredmények Összefoglalása

Táblázatos formában és ábraként is összefoglaljuk a modellek tanítási idejét és pontosságát.

In [ ]:
print("\n--- 1. Feladat: Összehasonlítás ---")
print(f"{'Modell':<25} {'Tanítási idő (s)':<20} {'Tanuló pontosság':<20} {'Teszt pontosság':<20}")
print("-" * 85)
table_data_part1 = []
model_names_part1 = list(results_part1.keys())
for name in model_names_part1:
    metrics = results_part1[name]
    print(f"{name:<25} {training_times_part1[name]:<20.4f} {metrics['accuracy_train']:<20.4f} {metrics['accuracy_test']:<20.4f}")
    table_data_part1.append([f"{training_times_part1[name]:.4f}",
                           f"{metrics['accuracy_train']:.4f}",
                           f"{metrics['accuracy_test']:.4f}"])

# Táblázat készítése ábraként
fig, ax = plt.subplots(figsize=(10, max(1, len(model_names_part1) * 0.5)))
ax.axis('tight')
ax.axis('off')
columns = ['Tanítási idő (s)', 'Tanuló pontosság', 'Teszt pontosság']
the_table = ax.table(cellText=table_data_part1,
                     rowLabels=model_names_part1,
                     colLabels=columns,
                     loc='center',
                     cellLoc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(10)
the_table.scale(1.2, 1.2)
plt.title('1. Feladat - Modellek Összehasonlító Eredményei', y=1.1)
plt.tight_layout()
plt.savefig(os.path.join(output_dir_part1, 'summary_results_table.png'), bbox_inches='tight')
plt.show()

## 1.6 SVM Hiperparaméter-Hangolás ('C')

GridSearchCV segítségével megkeressük az SVM modell optimális `C` regularizációs paraméterét 5-szörös keresztvalidációval.

In [ ]:
print("\n--- 1. Feladat: Hiperparaméter-hangolás (SVM 'C') ---")
param_grid_svm_part1 = {'C': [0.01, 0.1, 1, 10, 100]}
svm_cv_part1 = SVC(random_state=random_state_part1, probability=True)

grid_search_part1 = GridSearchCV(svm_cv_part1, param_grid_svm_part1, cv=5, scoring='accuracy')
grid_search_part1.fit(X_train_scaled_part1, y_train_part1)

print(f"Legjobb 'C' paraméter: {grid_search_part1.best_params_['C']}")
print(f"Legjobb cross-validation pontosság: {grid_search_part1.best_score_:.4f}")

# Eredmények ábrázolása
cv_results_part1 = grid_search_part1.cv_results_
plt.figure(figsize=(8, 5))
plt.plot(param_grid_svm_part1['C'], cv_results_part1['mean_test_score'], marker='o')
plt.xscale('log')
plt.xlabel("'C' paraméter értéke")
plt.ylabel("Átlagos Cross-Validation Pontosság")
plt.title("1. Feladat - SVM Teljesítmény a 'C' paraméter függvényében")
plt.grid(True)
plt.savefig(os.path.join(output_dir_part1, 'svm_c_parameter_tuning.png'))
plt.show()

# Legjobb modell kiértékelése
best_svm_part1 = grid_search_part1.best_estimator_
y_pred_svm_best_part1 = best_svm_part1.predict(X_test_scaled_part1)
accuracy_svm_best_test_part1 = accuracy_score(y_test_part1, y_pred_svm_best_part1)
print(f"\nLegjobb SVM modell pontossága a teszt halmazon: {accuracy_svm_best_test_part1:.4f}")
print(f"  Legjobb SVM Támaszvektorok száma osztályonként: {best_svm_part1.n_support_}")
print("Osztályozási riport (Legjobb SVM, Teszt halmaz):")
print(classification_report(y_test_part1, y_pred_svm_best_part1))

## 1.7 Logisztikus Regresszió Iterációk Hatása

Megvizsgáljuk, hogyan befolyásolja a Logisztikus Regresszió `max_iter` paramétere a modell pontosságát.

In [ ]:
print("\n--- 1. Feladat: Logisztikus Regresszió Iterációk Hatása ---")
iterations_part1 = [1, 2, 3, 4, 5, 10, 20, 50, 100, 200, 500, 1000]
train_accuracies_lr_part1 = []
test_accuracies_lr_part1 = []

warnings.filterwarnings("ignore", category=ConvergenceWarning)
for n_iter in iterations_part1:
    lr_iter_part1 = LogisticRegression(random_state=random_state_part1, max_iter=n_iter, solver='lbfgs')
    lr_iter_part1.fit(X_train_scaled_part1, y_train_part1)
    train_acc = lr_iter_part1.score(X_train_scaled_part1, y_train_part1)
    test_acc = lr_iter_part1.score(X_test_scaled_part1, y_test_part1)
    train_accuracies_lr_part1.append(train_acc)
    test_accuracies_lr_part1.append(test_acc)
warnings.filterwarnings("default", category=ConvergenceWarning)

# Ábrázolás
plt.figure(figsize=(10, 6))
plt.plot(iterations_part1, train_accuracies_lr_part1, marker='o', label='Tanuló pontosság')
plt.plot(iterations_part1, test_accuracies_lr_part1, marker='x', linestyle='--', label='Teszt pontosság')
plt.xlabel("Iterációk száma (max_iter)")
plt.ylabel("Pontosság")
plt.title("1. Feladat - Logisztikus Regresszió Pontossága az Iterációk Függvényében")
plt.xscale('log')
plt.ylim(0.9, 1.02)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.savefig(os.path.join(output_dir_part1, 'logistic_regression_accuracy_vs_iterations.png'))
plt.show()

## 1.8 Döntési Határok Vizualizációja

Kirajzoljuk a modellek döntési határait a 2D PCA-val redukált adatokon, hogy vizuálisan lássuk, hogyan osztályoznak.

In [ ]:
print("\n--- 1. Feladat: Döntési határok vizualizációja (PCA adatokon) ---")

# PCA adatok felosztása
X_train_pca_part1, X_test_pca_part1, y_train_pca_part1, y_test_pca_part1 = train_test_split(
    X_pca_part1, y_part1, test_size=0.3, random_state=random_state_part1, stratify=y_part1
)

# Modellek újratanítása a 2D PCA adatokon
models_pca_part1 = {
    "Naive Bayes": GaussianNB(),
    "Support Vector Machine (Best C)": best_svm_part1, # Hangolt SVM
    "Logistic Regression": LogisticRegression(random_state=random_state_part1, max_iter=1000)
}

# Meshgrid létrehozása
h = .02
x_min, x_max = X_pca_part1[:, 0].min() - 1, X_pca_part1[:, 0].max() + 1
y_min, y_max = X_pca_part1[:, 1].min() - 1, X_pca_part1[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Ábra létrehozása
n_models_part1 = len(models_pca_part1)
fig_db_part1, axes_db_part1 = plt.subplots(1, n_models_part1, figsize=(n_models_part1 * 6, 5))
if n_models_part1 == 1:
    axes_db_part1 = [axes_db_part1]

all_handles_part1 = []
all_labels_part1 = []
class_labels_part1 = [f'Osztály {i}' for i in range(centers)]

for idx, (name, model_pca) in enumerate(models_pca_part1.items()):
    model_pca.fit(X_train_pca_part1, y_train_pca_part1)
    Z = model_pca.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax = axes_db_part1[idx]
    ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
    scatter_train = ax.scatter(X_train_pca_part1[:, 0], X_train_pca_part1[:, 1], c=y_train_pca_part1, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    scatter_test = ax.scatter(X_test_pca_part1[:, 0], X_test_pca_part1[:, 1], c=y_test_pca_part1, cmap=plt.cm.coolwarm, s=50, edgecolors='grey', marker='^')

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(name)

    if idx == 0:
        handles_train, _ = scatter_train.legend_elements()
        all_handles_part1.extend(handles_train)
        all_labels_part1.extend(class_labels_part1)
        all_handles_part1.append(scatter_test) # Handle for test points
        all_labels_part1.append('Teszt pontok')

fig_db_part1.legend(handles=all_handles_part1, labels=all_labels_part1, loc='center right', bbox_to_anchor=(1.1, 0.5))
fig_db_part1.suptitle("1. Feladat - Döntési határok a 2D PCA adatokon")
plt.tight_layout(rect=[0, 0, 0.9, 1])
plt.savefig(os.path.join(output_dir_part1, 'decision_boundaries_pca.png'), bbox_inches='tight')
plt.show()

print("\nAz 1. részfeladat elemzése befejeződött.")

# 2. Feladat: Osztályozás a Wine Adathalmazon

## 2.1 Adat Betöltése

Betöltjük a Wine adathalmazt az UCI Machine Learning Repository URL-jéről `pandas` segítségével, és megadjuk az oszlopneveket.

In [ ]:
url_part2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
column_names_part2 = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash',
                      'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
                      'Proanthocyanins', 'Color intensity', 'Hue',
                      'OD280/OD315 of diluted wines', 'Proline']

try:
    df_part2 = pd.read_csv(url_part2, header=None, names=column_names_part2)
    print("2. Feladat - Wine adathalmaz sikeresen betöltve.")
    print(f"Adathalmaz mérete: {df_part2.shape}")
    # print("Adathalmaz első 5 sora:")
    # display(df_part2.head()) # Use display in notebooks
    # print("\nInformáció az adathalmazról:")
    # df_part2.info()
    print("\nOsztályok eloszlása:")
    print(df_part2['Class'].value_counts())
except Exception as e:
    print(f"Hiba történt az adatok letöltése közben: {e}")

## 2.2 Adat Vizualizációja (PCA)

Az első feladathoz hasonlóan PCA-t alkalmazunk a Wine adathalmaz jellemzőire is a 2D vizualizációhoz.

In [ ]:
if 'df_part2' in locals(): # Csak akkor fut, ha az adatbetöltés sikeres volt
    X_viz_part2 = df_part2.drop('Class', axis=1)
    y_viz_part2 = df_part2['Class']

    scaler_pca_part2 = StandardScaler()
    X_viz_scaled_part2 = scaler_pca_part2.fit_transform(X_viz_part2)

    pca_part2 = PCA(n_components=2, random_state=42)
    X_pca_part2 = pca_part2.fit_transform(X_viz_scaled_part2)

    plt.figure(figsize=(8, 6))
    scatter_part2 = plt.scatter(X_pca_part2[:, 0], X_pca_part2[:, 1], c=y_viz_part2, cmap='viridis', edgecolor='k', s=50)
    plt.title('2. Feladat - Wine adathalmaz (PCA után)')
    plt.xlabel('Első főkomponens')
    plt.ylabel('Második főkomponens')
    handles, labels = scatter_part2.legend_elements()
    class_labels_part2 = [f'Osztály {i}' for i in sorted(y_viz_part2.unique())]
    plt.legend(handles=handles, labels=class_labels_part2)
    plt.grid(True)
    plt.savefig(os.path.join(output_dir_part2, 'wine_data_pca.png'))
    plt.show()

## 2.3 Adat Felosztása és Skálázása

Felosztjuk a Wine adatokat tanító és teszt halmazra, majd skálázzuk őket.

In [ ]:
if 'df_part2' in locals():
    X_part2 = df_part2.drop('Class', axis=1)
    y_part2 = df_part2['Class']

    X_train_part2, X_test_part2, y_train_part2, y_test_part2 = train_test_split(
        X_part2, y_part2, test_size=0.3, random_state=42, stratify=y_part2
    )

    scaler_model_part2 = StandardScaler()
    X_train_scaled_part2 = scaler_model_part2.fit_transform(X_train_part2)
    X_test_scaled_part2 = scaler_model_part2.transform(X_test_part2)

    print(f"2. Feladat - Tanító halmaz mérete: {X_train_scaled_part2.shape[0]}")
    print(f"2. Feladat - Teszt halmaz mérete: {X_test_scaled_part2.shape[0]}")

## 2.4 Modellek Tanítása és Kiértékelése

Tanítjuk és kiértékeljük a modelleket a Wine adathalmazon. Itt a k-NN modellt is hozzáadjuk.

In [ ]:
if 'df_part2' in locals():
    models_part2 = {
        "Naive Bayes": GaussianNB(),
        "Support Vector Machine": SVC(random_state=42, probability=True),
        "Logistic Regression": LogisticRegression(random_state=42, max_iter=10000),
        "k-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
    }

    results_part2 = {}
    training_times_part2 = {}

    print("\n--- 2. Feladat: Modellek Kiértékelése ---")

    for name, model in models_part2.items():
        print(f"\n--- {name} ---")
        start_time = time.time()
        model.fit(X_train_scaled_part2, y_train_part2)
        end_time = time.time()
        training_times_part2[name] = end_time - start_time

        y_pred_train_part2 = model.predict(X_train_scaled_part2)
        y_pred_test_part2 = model.predict(X_test_scaled_part2)

        accuracy_train_part2 = accuracy_score(y_train_part2, y_pred_train_part2)
        accuracy_test_part2 = accuracy_score(y_test_part2, y_pred_test_part2)
        report_test_part2 = classification_report(y_test_part2, y_pred_test_part2)
        cm_test_part2 = confusion_matrix(y_test_part2, y_pred_test_part2)

        results_part2[name] = {
            "model": model,
            "accuracy_train": accuracy_train_part2,
            "accuracy_test": accuracy_test_part2,
            "report_test": report_test_part2,
            "confusion_matrix_test": cm_test_part2
        }

        print(f"Tanítási idő: {training_times_part2[name]:.4f} másodperc")
        print(f"Pontosság (Tanító halmaz): {accuracy_train_part2:.4f}")
        print(f"Pontosság (Teszt halmaz): {accuracy_test_part2:.4f}")
        print("Osztályozási riport (Teszt halmaz):")
        print(report_test_part2)
        print("Konfúziós mátrix (Teszt halmaz):")

        # Konfúziós mátrix ábrázolása
        plt.figure(figsize=(6, 4))
        class_labels_sorted_part2 = sorted(y_part2.unique())
        sns.heatmap(cm_test_part2, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_labels_sorted_part2, yticklabels=class_labels_sorted_part2)
        plt.xlabel('Jósolt osztály')
        plt.ylabel('Valós osztály')
        plt.title(f'{name} - Konfúziós Mátrix (Teszt)')
        safe_name = name.replace(" ", "_").replace("-", "").replace("(", "").replace(")", "").lower()
        plt.savefig(os.path.join(output_dir_part2, f'confusion_matrix_{safe_name}.png'))
        plt.show()

## 2.5 Eredmények Összefoglalása

Összefoglaljuk a Wine adathalmazon elért eredményeket.

In [ ]:
if 'results_part2' in locals():
    print("\n--- 2. Feladat: Összehasonlítás ---")
    print(f"{'Modell':<25} {'Tanítási idő (s)':<20} {'Tanuló pontosság':<20} {'Teszt pontosság':<20}")
    print("-" * 85)
    table_data_part2 = []
    model_names_part2 = list(results_part2.keys())
    for name in model_names_part2:
        metrics = results_part2[name]
        print(f"{name:<25} {training_times_part2[name]:<20.4f} {metrics['accuracy_train']:<20.4f} {metrics['accuracy_test']:<20.4f}")
        table_data_part2.append([f"{training_times_part2[name]:.4f}",
                               f"{metrics['accuracy_train']:.4f}",
                               f"{metrics['accuracy_test']:.4f}"])

    # Táblázat készítése ábraként
    fig_table, ax_table = plt.subplots(figsize=(10, max(2, len(model_names_part2) * 0.5)))
    ax_table.axis('tight')
    ax_table.axis('off')
    columns_part2 = ['Tanítási idő (s)', 'Tanuló pontosság', 'Teszt pontosság']
    the_table_part2 = ax_table.table(cellText=table_data_part2,
                                     rowLabels=model_names_part2,
                                     colLabels=columns_part2,
                                     loc='center',
                                     cellLoc='center')
    the_table_part2.auto_set_font_size(False)
    the_table_part2.set_fontsize(10)
    the_table_part2.scale(1.2, 1.2)
    plt.title('2. Feladat - Modellek Összehasonlító Eredményei (Wine)', y=1.1, fontsize=12)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir_part2, 'summary_results_table_part2.png'), bbox_inches='tight')
    plt.show()

## 2.6 Logisztikus Regresszió Iterációk Hatása

Megvizsgáljuk a Logisztikus Regresszió konvergenciáját a Wine adathalmazon.

In [ ]:
if 'df_part2' in locals():
    print("\n--- 2. Feladat: Logisztikus Regresszió Iterációk Hatása ---")
    iterations_lr_part2 = [1, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
    train_accuracies_lr_part2 = []
    test_accuracies_lr_part2 = []

    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    for n_iter in iterations_lr_part2:
        lr_iter_part2 = LogisticRegression(random_state=42, max_iter=n_iter, solver='lbfgs')
        lr_iter_part2.fit(X_train_scaled_part2, y_train_part2)
        train_acc = lr_iter_part2.score(X_train_scaled_part2, y_train_part2)
        test_acc = lr_iter_part2.score(X_test_scaled_part2, y_test_part2)
        train_accuracies_lr_part2.append(train_acc)
        test_accuracies_lr_part2.append(test_acc)
    warnings.filterwarnings("default", category=ConvergenceWarning)

    # Ábrázolás
    plt.figure(figsize=(10, 6))
    plt.plot(iterations_lr_part2, train_accuracies_lr_part2, marker='o', label='Tanuló pontosság')
    plt.plot(iterations_lr_part2, test_accuracies_lr_part2, marker='x', linestyle='--', label='Teszt pontosság')
    plt.xlabel("Iterációk száma (max_iter)")
    plt.ylabel("Pontosság")
    plt.title("2. Feladat - Logisztikus Regresszió Pontossága az Iterációk Függvényében (Wine)")
    plt.xscale('log')
    plt.ylim(0.9, 1.02)
    plt.grid(True, which="both", ls="--")
    plt.legend()
    plt.savefig(os.path.join(output_dir_part2, 'logistic_regression_accuracy_vs_iterations_part2.png'))
    plt.show()

## 2.7 SVM Hiperparaméter-Hangolás ('C' és 'gamma')

GridSearchCV segítségével megkeressük az SVM modell optimális `C` és `gamma` paramétereit.

In [ ]:
if 'df_part2' in locals():
    print("\n--- 2. Feladat: Hiperparaméter-hangolás (SVM) ---")
    param_grid_svm_part2 = {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.1, 1],
        'kernel': ['rbf']
    }
    svm_cv_part2 = SVC(probability=True)

    grid_search_svm_part2 = GridSearchCV(estimator=svm_cv_part2,
                                       param_grid=param_grid_svm_part2,
                                       cv=5,
                                       scoring='accuracy',
                                       n_jobs=-1,
                                       verbose=1)

    print("GridSearchCV indítása SVM-re...")
    start_time_gs = time.time()
    grid_search_svm_part2.fit(X_train_scaled_part2, y_train_part2)
    end_time_gs = time.time()
    print(f"GridSearchCV befejezve. Idő: {end_time_gs - start_time_gs:.2f} másodperc")

    print(f"\nLegjobb paraméterek (SVM): {grid_search_svm_part2.best_params_}")
    print(f"Legjobb keresztvalidációs pontosság (SVM): {grid_search_svm_part2.best_score_:.4f}")

    # Legjobb modell kiértékelése
    best_svm_part2 = grid_search_svm_part2.best_estimator_
    y_pred_svm_best_part2 = best_svm_part2.predict(X_test_scaled_part2)
    accuracy_svm_best_test_part2 = accuracy_score(y_test_part2, y_pred_svm_best_part2)

    print(f"\nLegjobb (hangolt) SVM modell pontossága a teszt halmazon: {accuracy_svm_best_test_part2:.4f}")
    print(f"  Legjobb SVM Támaszvektorok száma osztályonként: {best_svm_part2.n_support_}")
    print("Osztályozási riport (Legjobb SVM, Teszt halmaz):")
    print(classification_report(y_test_part2, y_pred_svm_best_part2))

    # Konfúziós mátrix a legjobb SVM modellhez
    cm_svm_best_part2 = confusion_matrix(y_test_part2, y_pred_svm_best_part2)
    plt.figure(figsize=(6, 4))
    class_labels_sorted_part2 = sorted(y_part2.unique())
    sns.heatmap(cm_svm_best_part2, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels_sorted_part2, yticklabels=class_labels_sorted_part2)
    plt.xlabel('Jósolt osztály')
    plt.ylabel('Valós osztály')
    plt.title('Legjobb (hangolt) SVM - Konfúziós Mátrix (Teszt)')
    plt.savefig(os.path.join(output_dir_part2, 'confusion_matrix_svm_best_tuned.png'))
    plt.show()

## 2.8 SVM Hangolás Vizualizációja (Heatmap)

Heatmap segítségével ábrázoljuk, hogyan változott az SVM keresztvalidációs pontossága a `C` és `gamma` paraméterek különböző kombinációira.

In [ ]:
if 'grid_search_svm_part2' in locals():
    print("\n--- 2. Feladat: SVM Hiperparaméter-hangolás Vizualizációja ---")
    results_df_part2 = pd.DataFrame(grid_search_svm_part2.cv_results_)
    results_df_part2 = results_df_part2[['param_C', 'param_gamma', 'mean_test_score']]
    results_df_part2['mean_test_score'] = results_df_part2['mean_test_score'].astype(float)

    try:
        # A pivot megpróbálja kezelni a 'scale', 'auto' stringeket is oszlopcímkeként
        scores_part2 = results_df_part2.pivot(index='param_C', columns='param_gamma', values='mean_test_score')

        plt.figure(figsize=(10, 6))
        sns.heatmap(scores_part2, annot=True, fmt=".4f", cmap="viridis")
        plt.title('2. Feladat - SVM Keresztvalidációs Pontosság (Heatmap)')
        plt.xlabel('Gamma paraméter')
        plt.ylabel('C paraméter')
        plt.savefig(os.path.join(output_dir_part2, 'svm_tuning_heatmap.png'))
        plt.show()

    except ValueError as ve:
        print(f"\nHiba a heatmap készítésekor: {ve}")
        print("Lehetséges ok: A 'gamma' paraméter tartalmazott nem numerikus értékeket ('scale', 'auto').")
    except KeyError as ke:
         print(f"\nHiba a heatmap készítésekor: Hiányzó oszlop - {ke}")

## 2.9 Döntési Határok Vizualizációja

Kirajzoljuk a modellek döntési határait a Wine adathalmaz PCA-val redukált változatán.

In [ ]:
if 'df_part2' in locals() and 'grid_search_svm_part2' in locals():
    print("\n--- 2. Feladat: Döntési határok vizualizációja (PCA adatokon) ---")

    # PCA adatok felosztása
    X_train_pca_part2, X_test_pca_part2, y_train_pca_part2, y_test_pca_part2 = train_test_split(
        X_pca_part2, y_part2, test_size=0.3, random_state=42, stratify=y_part2
    )

    # Modellek PCA adatokhoz
    models_pca_part2 = {
        "Naive Bayes": GaussianNB(),
        "Best Tuned SVM": grid_search_svm_part2.best_estimator_, # Hangolt SVM
        "Logistic Regression": LogisticRegression(random_state=42, max_iter=10000)
    }

    # Meshgrid
    h = .02
    x_min, x_max = X_pca_part2[:, 0].min() - 1, X_pca_part2[:, 0].max() + 1
    y_min, y_max = X_pca_part2[:, 1].min() - 1, X_pca_part2[:, 1].max() + 1
    xx_part2, yy_part2 = np.meshgrid(np.arange(x_min, x_max, h),
                                     np.arange(y_min, y_max, h))

    # Ábra
    n_models_part2 = len(models_pca_part2)
    fig_db_part2, axes_db_part2 = plt.subplots(1, n_models_part2, figsize=(n_models_part2 * 6, 5))
    if n_models_part2 == 1:
        axes_db_part2 = [axes_db_part2]

    all_handles_part2 = []
    all_labels_part2 = []
    class_labels_pca_part2 = [f'Osztály {i}' for i in sorted(y_part2.unique())]

    for idx, (name, model_pca) in enumerate(models_pca_part2.items()):
        model_pca.fit(X_train_pca_part2, y_train_pca_part2)
        Z_part2 = model_pca.predict(np.c_[xx_part2.ravel(), yy_part2.ravel()])
        Z_part2 = Z_part2.reshape(xx_part2.shape)

        ax = axes_db_part2[idx]
        ax.contourf(xx_part2, yy_part2, Z_part2, cmap=plt.cm.viridis, alpha=0.6)
        scatter_train_pca = ax.scatter(X_train_pca_part2[:, 0], X_train_pca_part2[:, 1], c=y_train_pca_part2,
                                      cmap=plt.cm.viridis, edgecolor='k', s=20)
        scatter_test_pca = ax.scatter(X_test_pca_part2[:, 0], X_test_pca_part2[:, 1], c=y_test_pca_part2,
                                     cmap=plt.cm.viridis, edgecolor='k', marker='^', s=30)

        ax.set_xlim(xx_part2.min(), xx_part2.max())
        ax.set_ylim(yy_part2.min(), yy_part2.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(name)

        if idx == 0:
            handles_train, _ = scatter_train_pca.legend_elements()
            all_handles_part2.extend(handles_train)
            all_labels_part2.extend(class_labels_pca_part2)
            all_handles_part2.append(scatter_test_pca) # Handle for test points
            all_labels_part2.append('Teszt pontok')

    fig_db_part2.legend(handles=all_handles_part2, labels=all_labels_part2, loc='lower center', ncol=len(all_labels_part2), bbox_to_anchor=(0.5, -0.05))
    fig_db_part2.suptitle('2. Feladat - Modellek Döntési Határai (PCA-redukált Wine Adatokon)', fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(output_dir_part2, 'decision_boundaries_pca_part2.png'), bbox_inches='tight')
    plt.show()

    print("\nAz 2. részfeladat elemzése befejeződött.")